In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import logging
import sys

logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s'
    , level=logging.INFO
    , datefmt = '%H:%M:%S'
    , stream = sys.stdout
)


from IPython.display import display,Markdown

from geoba import fillforms
import tosogoJupyter.rtkconf as pyconf
from gluecode import handlers as tosogo
from gluecode import printers as report
from gluecode.mdtemplates import *

# Memoria de Relevamiento

Se midieron con equipos GPS de simple frecuencia dos puntos que se denominaron
PFD1 y PFD2, y se vincularon a la estación permanente IGM1. En este apartado
se describe el procesamiento GPS, incluyendo la salida del programa de procesamiento. 

In [ ]:
#CeldaDeParametros
FILEFLD = 'proyecto'
OUTPUTFLD = 'output'
FORMFLD = 'forms'
TMPFLD = 'proyfiles'
BASE = 'igm10240.20o'
PIE = 'PFD10240.20o'
AZIMUT = 'PFD20240.20o'

In [ ]:
!mkdir -p {TMPFLD}

In [ ]:
display(Markdown(mdfiles.format(BASE=BASE,PIE=PIE,AZIMUT=AZIMUT)))

## Archivos de Sesiones

Cada vector se separó en dos sesiones de medición, utilizando la herramienta RinEdit, de GPSTk

In [ ]:
datos = {}

for i,k in zip([BASE, PIE, AZIMUT],['base','pie','azimut']):
    archivo = os.path.join(FILEFLD,i)
    f = tosogo.DataFile(archivo)
    datos[k] = f

REFNAV = datos['base'].path[:-1]+'n'

for k in ['pie','azimut','base']:
    datos[k].store_pdops(nav=REFNAV)

valid_eps_total = {'{}'.format(k):datos[k].pdop.size for k in ['pie','azimut']}

# TRIM PDOP > 2.5
for k in ['pie','azimut','base']:
    #datos[k].store_pdops(nav=REFNAV)
    datos[k] = datos[k]._pdop_filter()
    datos[k].store_pdops(nav=REFNAV)
    
valid_eps_filt = {'{}'.format(k):datos[k].pdop.size for k in ['pie','azimut']}

In [ ]:
display(Markdown(mdpdop.format(base=PIE[:4],az=AZIMUT[:4],
                               nbase=valid_eps_total['pie']-valid_eps_filt['pie'],
                               naz=valid_eps_total['azimut']-valid_eps_filt['azimut'],
                               nbasehs=valid_eps_filt['pie']/3600.,
                               nazhs=valid_eps_filt['azimut']/3600.)))

In [ ]:
SPLITPATH = os.path.join(TMPFLD,'splits')

!mkdir -p {SPLITPATH}

for k,v in datos.items():
    if k == 'base':
        continue
    else:
        v.mid_split(folder=SPLITPATH)
        #v.store_pdops(nav=REFNAV)

In [ ]:
display(Markdown(mdsessions.format(nbase=BASE[:4].upper(),npie=PIE[:4].upper(),naz=AZIMUT[:4].upper(),
                                  fpie=os.path.basename(datos['base'].path),
                                  fbase=os.path.basename(datos['pie'].path),
                                  faz=os.path.basename(datos['azimut'].path),
                                  fpie1=os.path.basename(datos['pie'].path).replace('0.','1.'),
                                  faz1=os.path.basename(datos['azimut'].path).replace('0.','1.'),
                                  fpie2=os.path.basename(datos['pie'].path).replace('0.','2.'),
                                  faz2=os.path.basename(datos['azimut'].path).replace('0.','2.'),
                                  )))

In [ ]:
for k,v in datos.items():
    v.store_pdops(nav=REFNAV)

In [ ]:
display(Markdown(mdpdopgraphs))

In [ ]:
for doprover,dopbase,titulo in zip(
    [datos['pie'].p1pdop,datos['pie'].p2pdop,datos['azimut'].p1pdop,datos['azimut'].p2pdop],
    [datos['base'].pdop,datos['base'].pdop,datos['pie'].pdop,datos['pie'].pdop],
    ['Vector Largo Sesion A','Vector Largo Sesion B',
     'Vector Corto Sesion A','Vector Corto Sesion B']):
    
    fig,ax = plt.subplots(figsize=(10,3))
    doprover.plot(style='x',ax=ax,label='PDOP Rover')
    dopbasere = dopbase.reindex(doprover.index)
    dopbasere.plot(style='.',ax=ax,label='PDOP Base')
    ax.set_title(titulo + ' PDOP máximo: {}'.format(max([doprover.max(),dopbasere.max()])))
    ax.legend()
    ax.set_xlabel("Tiempo")
    ax.set_ylabel("PDOP")
    plt.show()

# Con esta información se generaron los Anexos 4 y 4.1

In [ ]:
filesinfo = fillforms.get_rx_data(datos['base'].path,
                                  datos['pie'].p1path,
                                  datos['pie'].p2path )
fillforms.make_template(filesinfo,template_file=f'{FORMFLD}/Anexo4.1.fdf',
                        out=f'{OUTPUTFLD}/Anexo4.1-{BASE[:4]}-{PIE[:4]}.fdf')

In [ ]:
kwargs = {
    'pdf_file':f'{FORMFLD}/Anexo4.1.pdf',
    'fdf_file':f'{OUTPUTFLD}/Anexo4.1-{BASE[:4]}-{PIE[:4]}.fdf',
    'out':f'{OUTPUTFLD}/Anexo4.1-{BASE[:4]}-{PIE[:4]}.pdf'
}

fillforms.apply_template(**kwargs)

In [ ]:
filesinfo = fillforms.get_rx_data(datos['pie'].path,
                                  datos['azimut'].p1path,
                                  datos['azimut'].p2path )
fillforms.make_template(filesinfo,template_file=f'{FORMFLD}/Anexo4.1.fdf',
                        out=f'{OUTPUTFLD}/Anexo4.1-{PIE[:4]}-{AZIMUT[:4]}.fdf')

In [ ]:
kwargs = {
    'pdf_file':f'{FORMFLD}/Anexo4.1.pdf',
    'fdf_file':f'{OUTPUTFLD}/Anexo4.1-{PIE[:4]}-{AZIMUT[:4]}.fdf',
    'out':f'{OUTPUTFLD}/Anexo4.1-{PIE[:4]}-{AZIMUT[:4]}.pdf'
}

fillforms.apply_template(**kwargs)

# Antenas:

Las antenas utilizadas, modelo NAP100 de Ashtech, que son las antenas originales del equipo Promark 3, no se encuentran en el archivo de información de antena del software de procesamiento RTKLIB, los parámetros utilizados, extraidos de la información que viene grabada en la antena, están a continuación y son agregados al archivo de antenas para poder realizar el cálculo.

El formato del archivo es el siguiente:
https://kb.igs.org/hc/en-us/articles/216104678-ANTEX-format-description

Los datos agregados son:
```      
                                                            START OF ANTENNA    
NAP100          NONE                                        TYPE / SERIAL NO    
FIELD               JJC                      0    02-DIC-20 METH / BY / # / DATE
     0.0                                                    DAZI                
     0.0  80.0   5.0                                        ZEN1 / ZEN2 / DZEN  
     1                                                      # OF FREQUENCIES    
CONVERTED FROM RELATIVE NGS ANTENNA CALIBRATIONS            COMMENT             
   G01                                                      START OF FREQUENCY  
      0.00      0.00     73.00                              NORTH / EAST / UP   
   G01                                                      END OF FREQUENCY    
                                                            END OF ANTENNA      
```      
                                                            



In [ ]:
antenaNAP100 = \
"""
                                                            START OF ANTENNA    
NAP100          NONE                                        TYPE / SERIAL NO    
FIELD               JJC                      0    02-DIC-20 METH / BY / # / DATE
     0.0                                                    DAZI                
     0.0  80.0   5.0                                        ZEN1 / ZEN2 / DZEN  
     1                                                      # OF FREQUENCIES    
CONVERTED FROM RELATIVE NGS ANTENNA CALIBRATIONS            COMMENT             
   G01                                                      START OF FREQUENCY  
      0.00      0.00     73.00                              NORTH / EAST / UP   
   G01                                                      END OF FREQUENCY    
                                                            END OF ANTENNA
EOF
"""
!cat /rtklib/data/igs05.atx - <<EOF > /rtklib/data/antenas.atx $antenaNAP100

In [ ]:
from importlib import reload
pyconf= reload(pyconf)

In [ ]:
config_form1 = pyconf.formulario()

In [ ]:
pyconf.load_conf('/rtklib/confs/L1-long-bl.cfg',config_form1)

In [ ]:
config_form1

In [ ]:
pyconf.save_conf(f'{TMPFLD}/conf.cfg',config_form1)

In [ ]:
with open(f'{TMPFLD}/conf.cfg','r') as f:
    lines = f.readlines()

params = {i.strip():j.strip() for i,j in [k.split('=') for k in lines] }

display(Markdown(mdconfigs.format(**params)))

In [ ]:
import gluecode.downloaders as dld

In [ ]:
sp3files = dld.getPreciseEphs(datos['base'],f'{TMPFLD}/sp3')

# Procesamiento con rnx2rtkp

El procesamiento fue realizado con el programa por linea de comandos de RTKlib, ```rnx2rtkp```  en dos corridas, directa ('*forward*') e invertida ('*backward*') del filtro de kallman para cada sesión.

A continuación el encabezado de la salida del programa, el resto de la salida, correspondiente a los resultados, se presenta mas adelante en forma de tablas y gráficas.

In [ ]:
_= datos['pie'].process(datos['base'],f'{TMPFLD}/conf.cfg',sp3=sp3files,nav=REFNAV)

In [ ]:
_= datos['pie'].reloadProcesed()

In [ ]:
proceso = datos['pie']
display(Markdown(mdpos.format(npto=PIE[:4],
                              ptlat=report.GEOBA_final_pos(proceso,proj=None).iloc[0]['latitud'],
                              ptlon=report.GEOBA_final_pos(proceso,proj=None).iloc[0]['longitud'],
                              tablaGK=report.GEOBA_final_pos(proceso).to_markdown(floatfmt=".3f"),
                              tablaLL=report.GEOBA_final_pos(proceso,proj='geograficas').to_markdown(floatfmt=".3f"))))

In [ ]:
report.plotSessionGraphs(datos['pie'].sessData,filtro=lambda x:report.filtroGauss(report.filtroQ(x)))

In [ ]:
sitio = tosogo.parse_marker(datos['pie']._sumary)

with open('/rtklib/data/ramsac.pos','a+') as f:
    print('{latitud:13.9f} {longitud:14.9f} {altura:9.3f}  {punto}  {punto}'\
          .format(**report.GEOBA_final_pos(datos['pie'],proj=None).iloc[0],punto=sitio),file=f)

# Vector de Orientación

In [ ]:
config_form2 = pyconf.formulario()

In [ ]:
pyconf.load_conf('/rtklib/confs/L1-short-bl.cfg',config_form2)

In [ ]:
config_form2

In [ ]:
pyconf.save_conf(f'{TMPFLD}/conf2.cfg',config_form2)

In [ ]:
with open(f'{TMPFLD}/conf2.cfg','r') as f:
    lines = f.readlines()

params = {i.strip():j.strip() for i,j in [k.split('=') for k in lines] }

display(Markdown(mdconfigs.format(**params)))

# Procesamiento con rnx2rtkp

El procesamiento fue realizado con el programa por linea de comandos de RTKlib, ```rnx2rtkp```  en dos corridas, directa ('*forward*') e invertida ('*backward*') del filtro de kallman para cada sesión.

A continuación el encabezado de la salida del programa, el resto de la salida, correspondiente a los resultados, se presenta mas adelante en forma de tablas y gráficas.

In [ ]:
_= datos['azimut'].process(datos['pie'],f'{TMPFLD}/conf2.cfg',sp3=sp3files,nav=REFNAV)

In [ ]:
_= datos['azimut'].reloadProcesed()

In [ ]:
proceso = datos['azimut']
display(Markdown(mdpos.format(npto=PIE[:4],
                              ptlat=report.GEOBA_final_pos(proceso,proj=None).iloc[0]['latitud'],
                              ptlon=report.GEOBA_final_pos(proceso,proj=None).iloc[0]['longitud'],
                              tablaGK=report.GEOBA_final_pos(proceso).to_markdown(floatfmt=".3f"),
                              tablaLL=report.GEOBA_final_pos(proceso,proj='geograficas').to_markdown(floatfmt=".3f"))))

In [ ]:
report.plotSessionGraphs(datos['azimut'].sessData,filtro=lambda x:report.filtroGauss(report.filtroQ(x)))

In [ ]:
pd.DataFrame({tosogo.parse_marker(datos[i]._sumary):report.finalPOS(datos[i])\
              for i in ('base','pie','azimut')}).to_csv(f'{OUTPUTFLD}/Resultados.csv')